In [212]:
import json
import re
import hashlib

import requests
from bs4 import BeautifulSoup

In [213]:
images = set()

In [214]:
user_agent = 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Mobile Safari/537.36'

In [215]:
session = requests.Session()
response = session.get('https://www.instagram.com/explore/tags/lightnings/')
with open('./temp/out.html', 'w') as f:
    f.write(response.text)

In [216]:
match = re.findall(r'"display_url":"[^"]*', response.text)

for image in match:
    images.add(image[15:])

In [217]:
len(images)

80

In [218]:
match = re.search(
    r"<script[^>]*>\s*window._sharedData\s*=\s*((?!<script>).*)\s*;\s*</script>",
    response.text,
)

In [219]:
sharedData = json.loads(match.group(1))

entry_data = sharedData['entry_data']
page_info = entry_data['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['page_info']
end_cursor = page_info['end_cursor']
var_id = entry_data['TagPage'][0]['graphql']['hashtag']['id']

rhx_gis = sharedData['rhx_gis']

csrf_token = sharedData['config']['csrf_token']

In [220]:
variables = {
            #'id': var_id,
            'after': end_cursor,
            'first': 10
}
variables = json.dumps(variables)

values = "%s:%s" % (rhx_gis, variables)
x_instagram_gis = hashlib.md5(values.encode('utf-8')).hexdigest()
    
settings = {
    "params": {
        "variables": variables
    }
}     
settings['headers'] = {
    'User-Agent': user_agent,
    "X-Requested-With": "XMLHttpRequest",
    'X-Instagram-GIS': x_instagram_gis,
    'User-Agent': user_agent,
    'access_token': csrf_token
}
session.cookies.set('csrftoken', csrf_token)

Cookie(version=0, name='csrftoken', value='EF5S2ZTlMpT7eA4y8cjl93FaN19UmqJm', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [221]:
response = session.get('https://www.instagram.com/explore/tags/lightnings/', **settings)

match = re.findall(r'"display_url":"[^"]*', response.text)
for image in match:
    images.add(image[15:])
len(images)

80

_sharedData = responce.text[responce.text.find('window._sharedData'):]
config = '{"' + _sharedData[_sharedData.find('config'):].split('}', maxsplit=1)[0] + '}}'
csrf_token = json.loads(config)['config']['csrf_token']

responce = requests.get('https://www.instagram.com/explore/tags/lightnings/', data={'csrfmiddlewaretoken':csrf_token})
with open('out_token.html', 'w') as f:
    f.write(responce.text)
soup = BeautifulSoup(responce.text, 'html.parser')
soup